In [1]:
%pip install langchain openai langchain_community langchain_openai huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: packa

In [2]:
import pandas as pd

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from tqdm.auto import tqdm
from langchain_core.prompts import ChatPromptTemplate

base_url = "http://saiga.vkcloud.eazify.net:8000/v1"

In [3]:
chat = ChatOpenAI(api_key="<key>",
                  model = "tgi",
                  openai_api_base = base_url,
                  max_tokens=700)

In [8]:
bert_tags = ['Украина', 'политика', 'Верховная рада', 'законы']
bert_tags_str = ', '.join(bert_tags)
bert_tags_str

'Украина, политика, Верховная рада, законы'

In [86]:
items

{'LOC': ['Президент', 'Швейцарииии', 'Швейцарии', 'США'],
 'PER': ['Виола Амхерд', 'Владимиру Путину'],
 'ORG': ['Blick',
  'Минобороны Швейцаром в',
  '«Крокус Сити Холле',
  'рабочей группе по поиску и конфискации замороженных российских активов']}

In [87]:
for i in items:
  print(i)

LOC
PER
ORG


In [88]:
ner_data = {
    'LOC': ['Президент', 'Швейцарииии', 'Швейцарии', 'США'],
    'PER': ['Виола Амхерд', 'Владимиру Путину'],
    'ORG': ['Blick', 'Минобороны Швейцаром в', '«Крокус Сити Холле', 'рабочей группе по поиску и конфискации замороженных российских активов']
}

ner_str = ""
for key, lst in ner_data:
  for elem in lst:
    ner_str += f"{elem} (сущность {key}), "
ner_str = ner_str[:-2]

ner_str

ValueError: too many values to unpack (expected 2)

In [6]:
ner_kws = ""

bert_tags = ['Украина', 'политика', 'Верховная рада', 'законы']
bert_tags_str = ', '.join(bert_tags)

xgboost_tags = ['политика', 'законы', 'экономика']
xgboost_tags_str = ', '.join(xgboost_tags)

ner_str = ""
for key, lst in items:
  for elem in lst:
    ner_str += f"{elem} (сущность {key}), "
ner_str = ner_str[:-2]

ner_results = [] # dict
ner_results_lst = ner_results.items() # -> [(key1, kwor1)]



bertopic = ['']
#bertopic_str = ', '.join(bertopic)

instruct = """
У меня есть тема, которая содержит следующие документы:
{text}
Тема описывается следующими ключевыми словами: {keywords}

На основе приведенной выше информации извлеките короткое, но очень описательное название темы, состоящее не более чем из 5 слов.
Убедитесь, что оно имеет следующий формат:
тема: <topic label>
"""

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", """Ты редактор новостного агенства, и твоя задача извлекать тэги из новостей."""),
        ("human", instruct),
    ]
)

topic = "1. Создание преступного сообщества (преступной организации) в целях совершения одного или нескольких тяжких или особо тяжких преступлений либо руководство преступным сообществом (преступной организацией) или входящими в него (нее) структурными подразделениями,  а равно координация действий организованных групп, создание устойчивых связей между ними, разработка планов и создание условий для совершения преступлений  организованными группами, раздел сфер преступного влияния и (или) преступных доходов между такими группами - наказываются лишением свободы на срок от двенадцати до двадцати лет со штрафом в размере до пяти миллионов рублей или в размере заработной платы или иного дохода осужденного за период до пяти лет либо без такового и с ограничением свободы на срок от одного года до двух лет. (часть 1 в ред. Федерального закона от 01.04.2019 N 46-ФЗ) (см. текст в предыдущей редакции)"

news = "На Украине захотели принять законопроект, который вводит военную подготовку в школах. Об этом пишет ТАСС со ссылкой на украинское издание «Судебно-юридическая газета». Комитет по вопросам молодежи и спорта Верховной рады рекомендовал украинскому парламенту принять за основу соответствующий законопроект."
kws = "политика, Россия, Украина"

res = chat.invoke(chat_template.format_messages(text=news, keywords=kws))
print(res.content)

тема: Политика на Украине


In [35]:
d = {
    'LOC': ['Президент', 'Швейцарииии', 'Швейцарии', 'США'],
    'PER': ['Виола Амхерд', 'Владимиру Путину'],
    'ORG': ['Blick', 'Минобороны Швейцаром в', '«Крокус Сити Холле', 'рабочей группе по поиску и конфискации замороженных российских активов']
}



In [15]:
"президент (сущность LOC)"

items = d.items()
items

dict_items([('LOC', ['Президент', 'Швейцарииии', 'Швейцарии', 'США']), ('PER', ['Виола Амхерд', 'Владимиру Путину']), ('ORG', ['Blick', 'Минобороны Швейцаром в', '«Крокус Сити Холле', 'рабочей группе по поиску и конфискации замороженных российских активов'])])

In [40]:
#-------------------------------------------------------------------------

ans = ""
for key, lst in items:
  for elem in lst:
    ans += f"{elem} (сущность {key}), "
ans = ans[:-2]
ans

'Президент (сущность LOC), Швейцарииии (сущность LOC), Швейцарии (сущность LOC), США (сущность LOC), Виола Амхерд (сущность PER), Владимиру Путину (сущность PER), Blick (сущность ORG), Минобороны Швейцаром в (сущность ORG), «Крокус Сити Холле (сущность ORG), рабочей группе по поиску и конфискации замороженных российских активов (сущность ORG)'

In [37]:
v_dict, k_dict = d.values(), d.keys()
v_dict
k_dict

dict_keys(['LOC', 'PER', 'ORG'])

In [24]:
ans = ""
for elem in range(len(values)):
  joiner = f"(сущность {key}), "
  ans += joiner.join(list(map(str, values)))
ans

"['Президент', 'Швейцарииии', 'Швейцарии', 'США'](сущность ORG), ['Виола Амхерд', 'Владимиру Путину'](сущность ORG), ['Blick', 'Минобороны Швейцаром в', '«Крокус Сити Холле', 'рабочей группе по поиску и конфискации замороженных российских активов']['Президент', 'Швейцарииии', 'Швейцарии', 'США'](сущность ORG), ['Виола Амхерд', 'Владимиру Путину'](сущность ORG), ['Blick', 'Минобороны Швейцаром в', '«Крокус Сити Холле', 'рабочей группе по поиску и конфискации замороженных российских активов']['Президент', 'Швейцарииии', 'Швейцарии', 'США'](сущность ORG), ['Виола Амхерд', 'Владимиру Путину'](сущность ORG), ['Blick', 'Минобороны Швейцаром в', '«Крокус Сити Холле', 'рабочей группе по поиску и конфискации замороженных российских активов']"

In [39]:
print(v_dict, k_dict)
ans = ""
for elem in range(len(v_dict)):
  joiner = f"(сущность {k_dict[elem]}), "
  ans += joiner.join(map(str, v_dict[elem]))
ans

dict_values([['Президент', 'Швейцарииии', 'Швейцарии', 'США'], ['Виола Амхерд', 'Владимиру Путину'], ['Blick', 'Минобороны Швейцаром в', '«Крокус Сити Холле', 'рабочей группе по поиску и конфискации замороженных российских активов']]) dict_keys(['LOC', 'PER', 'ORG'])


TypeError: 'dict_keys' object is not subscriptable

In [83]:
ans = ""
for key, lst in items:
  for elem in lst:
    ans += f"{elem} (сущность {key}), "
ans = ans[:-2]
ans


'Президент (сущность LOC), Швейцарииии (сущность LOC), Швейцарии (сущность LOC), США (сущность LOC), Виола Амхерд (сущность PER), Владимиру Путину (сущность PER), Blick (сущность ORG), Минобороны Швейцаром в (сущность ORG), «Крокус Сити Холле (сущность ORG), рабочей группе по поиску и конфискации замороженных российских активов (сущность ORG)'

In [82]:
result = ", ".join(f"{[i + ' (сущность ' + key.capitalize() + ')' for i in value]}" for key, value in d.items())
#type(result)

# result = result.replace("[", "")
# result = result.replace("]", "")

print(result)

['Президент (сущность Loc)', 'Швейцарииии (сущность Loc)', 'Швейцарии (сущность Loc)', 'США (сущность Loc)'], ['Виола Амхерд (сущность Per)', 'Владимиру Путину (сущность Per)'], ['Blick (сущность Org)', 'Минобороны Швейцаром в (сущность Org)', '«Крокус Сити Холле (сущность Org)', 'рабочей группе по поиску и конфискации замороженных российских активов (сущность Org)']
